# Расчёт юнит экономики привлечения клиентов из разных каналов
Бизнесу необходимо произвести рассчёты эффективности вложения средств в разные каналы привлечения клиентов.

## Цели расчётов:
1. Определить какие каналы окупаются;
2. Определить какой из каналов самый прибыльный с точки зрения прибыли на один юнит;
3. Сформулировать бизнес выводы на основании данных расчётов.

## Ход расчётов:
Данные о LTV пользователей и стоимости привлечения находятся в файлах `customers.csv` и `cost_attraction.csv`. Перед выполнением расчётов необходимо произвести обзор данных. В предобработке данных будет произведена проверка на наличие ошибок в данных и при их наличии оценка их влияния на рассчёты. После этого будут осуществлены расчёты и формирование бизнес выводов.

В файле `customers.csv` содержится статистика по клиентам, привлеченным из каналов.
Поля:
- id клиента - номер клиента;
- Канал привлечения - канал из которого пришел клиент;
- LTV клиента, руб. - сколько прибыли принёс клиент за всё время взаимодействия с компанией.

В файле `cost_attraction.csv` содержится информация о общей стоимости привлечения для каждого канала.
Поля:
- Канал привлечения - канал, на который выполнены расходы на привлечение;
- Стоимость привлечения, руб. - сколько всего было потрачено на привлечение в данном канале.

## Стадии проекта:
1. Обзор данных
2. Предобработка данных
3. Расчёты
4. Бизнес выводы

# 1. Обзор данных
### Обзор таблиц:

In [94]:
import pandas as pd
# сохраняем в df данные по LTV пользователей
df_customers = pd.read_csv('data/customers.csv', sep = ';')
# сохраняем в df данные по затратам на привлечение в каждом канале
df_cost_attr = pd.read_csv('data/cost_attraction.csv', sep = ';')

Обзор данных по пользователям из таблицы `customers.csv`

In [95]:
# выведем первые 10 пользователей
df_customers.head(10)

,id клиента,Канал привлечения,"LTV, руб."
0,1,Реклама у блогеров,90
1,2,Реклама в новостных СМИ,120
2,3,Реклама в новостных СМИ,240
3,4,Контекстная реклама,590
4,5,Контекстная реклама,670
5,6,Реклама у блогеров,80
6,7,Реклама в новостных СМИ,100
7,8,Реклама в новостных СМИ,120
8,9,Реклама в новостных СМИ,300
9,10,Реклама в новостных СМИ,370


Обзор данных по стоимости привлечения в каналах из таблицы `cost_attraction.csv`

In [96]:
# выведем данные по стоимости
df_cost_attr

,Канал привлечения,"Стоимость привлечения, руб."
0,Таргетная реклама,100000
1,e-mail рассылки,25000
2,Реклама у блогеров,120000
3,Контекстная реклама,90000
4,Нативная реклама,70000
5,Реклама в новостных СМИ,60000


### Общая информация о таблицах:

In [97]:
# информация о таблице `customers.csv`
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1801 entries, 0 to 1800
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id клиента         1801 non-null   int64 
 1   Канал привлечения  1801 non-null   object
 2   LTV, руб.          1801 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 42.3+ KB


In [98]:
# информация о таблице `cost_attraction.csv`
df_cost_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Канал привлечения            6 non-null      object
 1   Стоимость привлечения, руб.  6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


Мы видим что в таблицах `customers.csv` и `cost_attraction.csv`:
- пропусков данных нет
- данные в полях в подходящих типах
- имена полей подходят нашим требованиям

# 2. Предобработка данных

### Проверка числовых значений:

Выполним проверку числовых значений в таблице `customers.csv` в поле `LTV, руб.` с целью исключения ошибок в данных. Отсортируем значения, так как при сортировке показываются ошибки в данных.

In [99]:
# отсортируем значения в df по полю `LTV, руб.` по возростанию
df_sorted = df_customers.sort_values(by='LTV, руб.', ascending=True)
# первые 5 элементов
df_sorted.head()

,id клиента,Канал привлечения,"LTV, руб."
1800,1801,Таргетная реклама,-100000
946,947,Реклама у блогеров,70
1020,1021,Реклама у блогеров,70
1539,1540,Реклама у блогеров,70
1051,1052,Реклама у блогеров,70


In [100]:
# последние 5 элементов
df_sorted.tail()

,id клиента,Канал привлечения,"LTV, руб."
734,735,Контекстная реклама,790
1070,1071,Контекстная реклама,790
339,340,Контекстная реклама,790
1515,1516,Контекстная реклама,790
1403,1404,Контекстная реклама,790


Мы видим, что в строке с минимальным значением `LTV, руб.` равно `-100000` - это ошибка в данных и удалим данную строку. 

In [101]:
# пересохраним в df с данными по клиентам все строки, кроме строки с ошибочным значением
df_customers = df_customers[df_customers['LTV, руб.'] != -100000]
# выполним повторную сортировку для проверки
df_customers = df_customers.sort_values(by='LTV, руб.', ascending=True)
# выведем повторно для проверки удаления ошибочного значения
df_customers.head()

,id клиента,Канал привлечения,"LTV, руб."
1368,1369,Реклама у блогеров,70
1478,1479,Реклама у блогеров,70
1630,1631,Реклама у блогеров,70
1051,1052,Реклама у блогеров,70
946,947,Реклама у блогеров,70


### Проверка на дубликаты:

Проверим на уникальность пользователей в поле `id клиента` таблицы `customers.csv`

In [102]:
# кол-во уникальных пользователей
df_customers['id клиента'].nunique()

1800

Все 1800 пользователей уникальные.

Проверим на уникальность названия каналов в поле `канал привлечения` таблицы `customers.csv`

In [103]:
# кол-во уникальных пользователей
df_customers['Канал привлечения'].nunique()

6

Выведем уникальные названия каналов

In [104]:
# уникальные значения в поле
df_customers['Канал привлечения'].unique()

array(['Реклама у блогеров', 'Реклама в новостных СМИ', 'e-mail рассылки',
       'Нативная реклама', 'Таргетная реклама', 'Контекстная реклама'],
      dtype=object)

Все названия каналов уникальные.

# 3. Расчёты

### Создаём сводную таблицу по каналам привлечения клиентов с добавлением двух полей:
1. Средний LTV на канал в руб.
2. Кол-во клипентов, пришедших из канала

In [ ]:
# создаём сводную таблицу
df_agg = df_customers.groupby(['Канал привлечения',]).agg(
	mean = ('LTV, руб.', lambda x: round(x.mean(), 0)),  # подсчёт с округлением
	count = ('LTV, руб.', 'count'),                      # подсчёт без округления
	).reset_index()

df_agg.rename(columns={'mean': 'Средний LTV, руб.', 'count': 'Кол-во клиентов'}, inplace=True)

df_agg

,Канал привлечения,"Средний LTV, руб.",Кол-во клиентов
0,e-mail рассылки,235.0,100
1,Контекстная реклама,652.0,150
2,Нативная реклама,393.0,250
3,Реклама в новостных СМИ,236.0,300
4,Реклама у блогеров,216.0,700
5,Таргетная реклама,496.0,300
